In [ ]:
from scipy.signal import savgol_filter

# 平滑
x_smooth = savgol_filter(x, window_length=11, polyorder=3)

# 一階導數
dx = savgol_filter(x, 11, 3, deriv=1, delta=1/fs)

# 二階導數（加速度）
ddx = savgol_filter(x, 11, 3, deriv=2, delta=1/fs)


1. 先用 SG 濾波（保留形狀、避免破壞振動頻率）
2. 用 SG 求速度、加速度
3. 對加速度用 Butterworth 低通（把二階導數放大的高頻 noise 清掉）